### SILVERSURFBOARDS WEBSCRAP - OUTLET

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from jproperties import Properties
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os

In [2]:
# load the SMTP credentials; 

use_mail = True

try:
    with open ('./dockerpy/app.properties', 'r+b') as f:
        
        p = Properties()
        p.load(f, 'utf-8')

        user = p.get('mail').data
        password = p.get('password').data

except: 
    print('e-mail sender will not be used. File "app.properties" not found')
    use_mail = False

In [ ]:
driver = webdriver.Firefox()

url = 'https://silversurf.com.br/categoria-produto/outlet-pranchas/'

driver.get(url)


html_content = driver.page_source

soup = BeautifulSoup(html_content, 'html.parser')

links = soup.find_all('a', href = True), #{'class': "woocommerce-LoopProduct-link"})
links = links[0]

The geckodriver version (0.35.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (136.0); currently, geckodriver 0.36.0 is recommended for firefox 136.*, so it is advised to delete the driver in PATH and retry


In [5]:
# regex in the attrs href to get the ones that match 'outlet-prancha'

boards_list = []
import re

for link in links:
    
    link_text = link.attrs['href']
    match = re.search('/produto/outlet-prancha', link_text)
    if match:
        boards_list.append(link)


In [6]:
# lets filter with RE - get only PU and 6'0 or 6'11
boards_list_filtered = []

for board in boards_list:
    
    label = board.attrs.get('aria-label')
    if not label == None:  # this removes the secundary link (which not contains the images as children)
            
        match = re.search("6'1|6'0", label)
        
        if match:
            match2 = re.search("PU", label, re.IGNORECASE)
            if match2: 
                boards_list_filtered.append(board)
    

<h3> Now We've our elegible surf boards for take a look/e-mail </h3>

In [ ]:
# how its gonna be the e-mail? - Title, price and image in the body of the e-mail;

for candidate in boards_list_filtered:
    pics = []

    candidate_pics = list(candidate.children)

    try:
        candidate_pics.remove('\n')
        candidate_pics.remove(' ')
    except:
        None

    for child in candidate_pics:

        if child.attrs != {}:
    
            pics.append(f"<img src={child.get('data-lazy-src')}>")
    
    candidate['img_source'] = pics

<h3> E-mail Sender </h3>

In [ ]:
html_msg = ""

for candidate in boards_list_filtered:

    html_msg = html_msg + f"""
    <html>
        <body>
        {candidate.attrs.get('aria-label')}<br>
        link da loja: {candidate.attrs.get('href')}<br>
        images:<br>
        {''.join(candidate.attrs.get('img_source'))}
        <br>
        <br>
        <br>
        </body>
    </html>
    """


# variables to send mail
if use_mail:
    
    msg = MIMEMultipart("alternative")
    msg['Subject'] = "Prancha em destaque"
    msg['From'] = "farialimer.green@gmail.com"
    msg['To'] = "farialimer.green@gmail.com"
    msg.attach(MIMEText(html_msg, 'html'))

    # Send email via SMTP (example with Gmail)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(user, password)
        server.send_message(msg)

else: # saves the msg as .html and open it in firefox browser; 

    with open('msg.html', 'w') as m:
        m.write(html_msg)

    msg_abspath = os.path.abspath('msg.html')
    driver.get('file://' + msg_abspath)

In [ ]:
# close the webdriver
driver.close()

# --- finish